In [28]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pathlib
import glob
import itertools
import pandas as pd
#import splitfolders
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import classification_report
from mlxtend.plotting import plot_confusion_matrix

In [ ]:
train_data_dir = r"D:/experiments/Datasets/ip102_v1.1-002/ip102_v1.1/prepared_data/train_all/"
valid_data_dir = r"D:/experiments/Datasets/ip102_v1.1-002/ip102_v1.1/prepared_data/val_all/"
test_data_dir = r"D:/experiments/Datasets/ip102_v1.1-002/ip102_v1.1/prepared_data/test_all/"

In [13]:
classes = ['rice leaf roller',
        'rice leaf caterpillar',
        'paddy stem maggot',
        'asiatic rice borer',
        'yellow rice borer',
        'rice gall midge',
        'Rice Stemfly',
        'brown plant hopper',
        'white backed plant hopper',
        'small brown plant hopper',
        'rice water weevil',
        'rice leafhopper',
        'grain spreader thrips',
        'rice shell pest',
        'grub',
        'mole cricket',
        'wireworm',
        'white margined moth',
        'black cutworm',
        'large cutworm',
        'yellow cutworm',
        'red spider',
        'corn borer',
        'army worm',
        'aphids',
        'Potosiabre vitarsis',
        'peach borer',
        'english grain aphid',
        'green bug',
        'bird cherry-oataphid',
        'wheat blossom midge',
        'penthaleus major',
        'longlegged spider mite',
        'wheat phloeothrips',
        'wheat sawfly',
        'cerodonta denticornis',
        'beet fly',
        'flea beetle',
        'cabbage army worm',
        'beet army worm',
        'Beet spot flies',
        'meadow moth',
        'beet weevil',
        'sericaorient alismots chulsky',
        'alfalfa weevil',
        'flax budworm',
        'alfalfa plant bug',
        'tarnished plant bug',
        'Locustoidea',
        'lytta polita',
        'legume blister beetle',
        'blister beetle',
        'therioaphis maculata Buckton',
        'odontothrips loti',
        'Thrips',
        'alfalfa seed chalcid',
        'Pieris canidia',
        'Apolygus lucorum',
        'Limacodidae',
        'Viteus vitifoliae',
        'Colomerus vitis',
        'Brevipoalpus lewisi McGregor',
        'oides decempunctata',
        'Polyphagotars onemus latus',
        'Pseudococcus comstocki Kuwana',
        'parathrene regalis',
        'Ampelophaga',
        'Lycorma delicatula',
        'Xylotrechus',
        'Cicadella viridis',
        'Miridae',
        'Trialeurodes vaporariorum',
        'Erythroneura apicalis',
        'Papilio xuthus',
        'Panonchus citri McGregor',
        'Phyllocoptes oleiverus ashmead',
        'Icerya purchasi Maskell',
        'Unaspis yanonensis',
        'Ceroplastes rubens',
        'Chrysomphalus aonidum',
        'Parlatoria zizyphus Lucus',
        'Nipaecoccus vastalor',
        'Aleurocanthus spiniferus',
        'Tetradacus c Bactrocera minax',
        'Dacus dorsalis(Hendel)',
        'Bactrocera tsuneonis',
        'Prodenia litura',
        'Adristyrannus',
        'Phyllocnistis citrella Stainton',
        'Toxoptera citricidus',
        'Toxoptera aurantii',
        'Aphis citricola Vander Goot',
        'Scirtothrips dorsalis Hood',
        'Dasineura sp',
        'Lawana imitata Melichar',
        'Salurnis marginella Guerr',
        'Deporaus marginatus Pascoe',
        'Chlumetia transversa',
        'Mango flat beak leafhopper',
        'Rhytidodera bowrinii white',
        'Sternochetus frigidus',
        'Cicadellidae'
]

# ResNet50

In [14]:
datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [15]:
# datagen = ImageDataGenerator()

In [16]:
train_batchs_Res = datagen.flow_from_directory(directory=train_data_dir, target_size=(224,224), classes=classes, batch_size=64)

valid_batchs_Res = datagen.flow_from_directory(directory=valid_data_dir, target_size=(224,224), classes=classes, batch_size=64)

test_batchs_Res = datagen.flow_from_directory(directory=test_data_dir, target_size=(224,224), classes=classes, batch_size=64, shuffle=False)

Found 38400 images belonging to 102 classes.
Found 6352 images belonging to 102 classes.
Found 20047 images belonging to 102 classes.


In [17]:
base_model_Res = ResNet50(include_top = False, weights = 'imagenet')

In [18]:
x = base_model_Res.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)

predictions = Dense(train_batchs_Res.num_classes, activation='softmax')(x)

modelRes = Model(inputs=base_model_Res.input, outputs=predictions)

for layer in base_model_Res.layers:
    layer.trainable = True

In [19]:
modelRes.compile(optimizer = Adam(learning_rate=0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [20]:
callbacks = [
        ModelCheckpoint("SavedModels/resnet_50_model/model_resnet50_aug.h5", verbose=1, save_best_model=True),
        ReduceLROnPlateau(monitor="val_loss", patience=3, factor=0.1, verbose=1, min_lr=1e-6),
        EarlyStopping(monitor="val_loss", patience=5, verbose=1)
    ]

In [21]:
history_Res = modelRes.fit(x = train_batchs_Res, 
            steps_per_epoch=train_batchs_Res.samples // 64,
            validation_data = valid_batchs_Res,
            validation_steps=valid_batchs_Res.samples // 64,
            epochs = 500, verbose = 1,
            callbacks=callbacks,)

Epoch 1/500
600/600 [==============================] - ETA: 0s - loss: 1.8775 - accuracy: 0.5366
Epoch 1: saving model to SavedModels/resnet_50_model\model_resnet50_aug.h5
600/600 [==============================] - 366s 602ms/step - loss: 1.8775 - accuracy: 0.5366 - val_loss: 3.8647 - val_accuracy: 0.1618 - lr: 1.0000e-04
Epoch 2/500
600/600 [==============================] - ETA: 0s - loss: 1.1279 - accuracy: 0.6857
Epoch 2: saving model to SavedModels/resnet_50_model\model_resnet50_aug.h5
600/600 [==============================] - 206s 343ms/step - loss: 1.1279 - accuracy: 0.6857 - val_loss: 1.4441 - val_accuracy: 0.6158 - lr: 1.0000e-04
Epoch 3/500
600/600 [==============================] - ETA: 0s - loss: 0.8770 - accuracy: 0.7448
Epoch 3: saving model to SavedModels/resnet_50_model\model_resnet50_aug.h5
600/600 [==============================] - 207s 344ms/step - loss: 0.8770 - accuracy: 0.7448 - val_loss: 1.2715 - val_accuracy: 0.6687 - lr: 1.0000e-04
Epoch 4/500
600/600 [=======

In [22]:
Res_model = tf.keras.models.load_model('SavedModels/resnet_50_model/model_resnet50_aug.h5')

Res_predictions = Res_model.predict(x=test_batchs_Res, verbose=0)

In [38]:
y_pred_Res = np.argmax(Res_predictions, axis=1)
y_true_Res = test_batchs_Res.classes

precision = precision_score(y_true_Res, y_pred_Res, average='weighted', zero_division=0)
print('Precision: %f' % precision)

recall = recall_score(y_true_Res, y_pred_Res, average='weighted', zero_division=0)
print('Recall: %f' % recall)

f1 = f1_score(y_true_Res, y_pred_Res, average='weighted', zero_division=0)
print('F1 score: %f' % f1)

accuracy = accuracy_score(y_true_Res, y_pred_Res)
print('Accuracy: %f' % accuracy)

Precision: 0.713208
Recall: 0.714571
F1 score: 0.708687
Accuracy: 0.714571


In [ ]:
print(classification_report(y_true_Res, y_pred_Res, target_names = classes, zero_division=0))

                                 precision    recall  f1-score   support

               rice leaf roller       0.62      0.81      0.70       226
          rice leaf caterpillar       0.52      0.50      0.51       110
              paddy stem maggot       0.47      0.17      0.25        42
             asiatic rice borer       0.63      0.62      0.63       207
              yellow rice borer       0.67      0.70      0.68       116
                rice gall midge       0.81      0.76      0.79       110
                   Rice Stemfly       0.48      0.61      0.54        61
             brown plant hopper       0.64      0.46      0.54       186
      white backed plant hopper       0.60      0.62      0.61       164
       small brown plant hopper       0.54      0.49      0.52        91
              rice water weevil       0.75      0.75      0.75       171
                rice leafhopper       0.42      0.47      0.45       109
          grain spreader thrips       0.58      0.

c:\Users\33735203\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\33735203\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\33735203\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [40]:
#Get the filename including last folder
image_names = [os.path.basename(filename) for filename in test_batchs_Res.filenames]


# Get the image names from the ImageDataGenerator
image_names = test_batchs_Res.filenames

df = pd.DataFrame({'Image Name': image_names, 'Predicted': y_pred_Res, 'Ground Truth': y_true_Res})
df.to_csv('results_R50_cls_aug.csv', index=False)

In [ ]:
matRes = confusion_matrix(y_true_Res,y_pred_Res)
plot_confusion_matrix(matRes, figsize=(36,36), class_names=classes, show_normed=True)

(<Figure size 3600x3600 with 1 Axes>,
 <Axes: xlabel='predicted label', ylabel='true label'>)